# E91 Implementation

1. create bell pairs
2. choose measurement for alice, bob
    * rotate $|0\rangle$, $|1\rangle$ by $a \in \{0,\frac{\pi}{4},\frac{\pi}{2}\}$ or $b \in \{\frac{\pi}{4},\frac{\pi}{2},\frac{3\pi}{4}\}$

Source: [https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.67.661](https://journals.aps.org/prl/pdf/10.1103/PhysRevLett.67.661)

In [40]:
from qiskit import QuantumCircuit, Aer, assemble
#from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint#, choice
import numpy as np
print("Imports Successful")

Imports Successful


In [41]:
msg_len = 1

In [42]:
# Prepare Bell State
qc = QuantumCircuit(2*msg_len, 2*msg_len)
for i in range(msg_len):
    q1 = 2*i
    q2 = 2*i + 1
    qc.x(q2)
    qc.h(q1)
    qc.cx(q1,q2)
qc.barrier()
qc.draw()

┌───┐      ░ 
q_0: ┤ H ├──■───░─
     ├───┤┌─┴─┐ ░ 
q_1: ┤ X ├┤ X ├─░─
     └───┘└───┘ ░ 
c_0: ═════════════
                  
c_1: ═════════════

In [43]:
def alice_measure(qc):
    chosen_bases = []
    for i in range(msg_len):
        alice_q = 2*i
        base = randint(3)
        chosen_bases.append(base)
        rot = np.pi * base / 4
        qc.u1(-rot,alice_q)
        qc.measure(alice_q,alice_q)
    return chosen_bases

def bob_measure(qc):
    chosen_bases = []
    for i in range(msg_len):
        bob_q = 2*i + 1
        base = randint(1,4)
        chosen_bases.append(base)
        rot = np.pi * base / 4
        qc.u1(-rot,bob_q)
        qc.measure(bob_q,bob_q)
    return chosen_bases

In [44]:
a_bases = alice_measure(qc)
b_bases = bob_measure(qc)

In [45]:
qasm_sim = Aer.get_backend('qasm_simulator')
qobj = assemble(qc, shots=1, memory=True)
result = qasm_sim.run(qobj).result()

In [46]:
result

namespace(backend_name='qasm_simulator',
          backend_version='0.5.2',
          qobj_id='bd96cdcc-c9fd-48ae-b488-a2de29caf6cf',
          job_id='0fa5f3df-e432-424b-9790-c407b16c49a0',
          success=True,
          results=[namespace(shots=1,
                             success=True,
                             data=namespace(memory=['0x2'],
                                            counts=namespace(0x2=1)),
                             meas_level=<MeasLevel.CLASSIFIED: 2>,
                             status='DONE',
                             header=namespace(clbit_labels=[['c', 0],
                                                            ['c', 1]],
                                              qreg_sizes=[['q', 2]],
                                              n_qubits=2,
                                              memory_slots=2,
                                              name='circuit14',
                                              creg_sizes=[['c', 2]],
